In [18]:
#import 

import pandas as pd
import geopandas as gpd
import folium
import numpy as np
from shapely import LineString, Polygon
from shapely.geometry import Point
import geopandas as gpd
from folium.plugins import HeatMap, HeatMapWithTime
import simplekml
import random
import shapely


In [19]:
#check how many grnd vehicles there are in the combined dataset
data=pd.read_csv('data/fr24/FR24_combined.csv')
data.drop('reserved',axis=1,inplace=True)

#DATA FILTERING
#
#getting all unique callsigns 
callsigns = pd.Series(data.callsign.unique(),name='callsigns')

#identify shorter callsigns related to tugs
callsign_mask = callsigns.str.len()<5
unique_id = callsigns[callsign_mask].unique().tolist()

#removing all known ground vehicles callsigns
mask = data['callsign'].isin(unique_id)
data = data[~mask]

#there are still some ground callsigns, that are from equip column
data = data[data.equip!='GRND']

#choosing only departing flights
data= data[data.schd_from =='AMS']

#filtering out flights we know nothing about (flight, callsign, equip)
data = data[~(data.callsign.isnull()&data.equip.isnull()&data.flight.isnull())]

#we can enrich the callsign by checking whether the flight column contains some info and copy it to the callsign column
data.loc[data.callsign.isnull() & ~data.flight.isnull()]['callsign']=data.loc[data.callsign.isnull() & ~data.flight.isnull()]['flight']

#replacing all other NaNs
data.fillna('unknown',inplace=True)

#dropping duplicates (if any)
data = data.drop_duplicates()

#dropping existing nans if any
data = data.dropna()
#dropping unnecessary columns
to_drop = ['snapshot_id','altitude','heading','radar_id','speed','squawk','reg','schd_from','schd_to','real_to','equip','flight']
data.drop(to_drop,inplace=True,axis=1)

data.reset_index(inplace=True,drop=True)

In [20]:
data.head()

,latitude,longitude,timestamp,flight_id,aircraft_id,callsign
0,52.30541,4.76520,2023-02-05 04:02:26,789932628,4739362,TRA303
1,52.30534,4.76499,2023-02-05 04:05:01,789932628,4739362,TRA303
2,52.30466,4.76480,2023-02-05 04:08:47,789932628,4739362,TRA303
3,52.30440,4.76507,2023-02-05 04:08:55,789932628,4739362,TRA303
4,52.30401,4.76545,2023-02-05 04:09:04,789932628,4739362,TRA303


In [21]:
#todo Check difference with geodesic distance using pyproj
#if too slow, then check vectorized with scipyafter converting to local UTM https://epsg.io/
#https://stackoverflow.com/questions/45805685/vectorization-to-calculate-many-distances
#https://stackoverflow.com/questions/68547631/shapely-linestring-total-length


def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    lat1.fillna(0,inplace=True)
    lon1.fillna(0,inplace=True)
    
    if to_radians:
        lat1 = np.radians(lat1)
        lon1 = np.radians(lon1)
        lat2 = np.radians(lat2)
        lon2 = np.radians(lon2)

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return (earth_radius * 2 * np.arcsin(np.sqrt(a)))

In [22]:
##TIME DIFF CALCULATION

# Convert the timestamp column to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Extract month, day, hour and time
data['month'] = data['timestamp'].dt.month
data['day'] = data['timestamp'].dt.day
data['hour'] = data['timestamp'].dt.hour
data['time'] = data['timestamp'].dt.strftime('%H:%M:%S')


# Extract the first 3 characters of callsign into a new "airline" column
data['airline'] =data['callsign'].str[:3]

# Remove the callsign column
data.drop(columns=['callsign'], inplace=True)

# Calculate time differences in seconds
data['time_difference'] = data['timestamp'].diff().dt.total_seconds()
data['time_difference'].fillna(0, inplace=True)  # Replace NaN with 0 for the first row

data = data.drop('timestamp',axis=1)

##DIStance DIFF CALCULATION

#calculate the distance
data['dist'] = haversine(data.latitude.shift(), data.longitude.shift(),data.loc[1:, 'latitude'], data.loc[1:, 'longitude'])

#filling in NaNs
data.dist.fillna(0,inplace=True)

In [23]:
data.head()

,latitude,longitude,flight_id,aircraft_id,month,day,hour,time,airline,time_difference,dist
0,52.30541,4.76520,789932628,4739362,2,5,4,04:02:26,TRA,0.0,0.000000
1,52.30534,4.76499,789932628,4739362,2,5,4,04:05:01,TRA,155.0,0.016262
2,52.30466,4.76480,789932628,4739362,2,5,4,04:08:47,TRA,226.0,0.076708
3,52.30440,4.76507,789932628,4739362,2,5,4,04:08:55,TRA,8.0,0.034247
4,52.30401,4.76545,789932628,4739362,2,5,4,04:09:04,TRA,9.0,0.050479


In [24]:
def transform(df,green_zone):
    
    """
    This function transforms the master dataframe (where one flight = multiple rows) into
    a geodataframe with one flight per row.  New columns are created:
    geometry:  the trajectory that is displayed on the map.
    max_time: max time difference between consecutive points
    length: total length along trajectory until the max_time difference
    decoup: location (lat, long) of the assumed decoupling position of the tug
    green: boolean column. True = if the decoupling position is inside the green zone. 
    day: day of the flight
    hour: hour of the flight
    month: month of the flight
    """
    
    geometries=list()
    keys=list()
    dates = list()
    decoup_loc=list()
    length = list()
    green =list()
    airline=list()
    hour=list()
    day=list()
    month=list()
    max_time_list = list()
    for key,grp in df.groupby('flight_id'):
        
        #keep index 0 constant
        grp.reset_index(inplace=True)
        #geometry development
        geometry = [Point(xy) for xy in zip(grp['latitude'], grp['longitude'])]
        
        #only if it is longer than a point we consider it a real geometry
        if len(geometry)>2:
            
            geometries.append(LineString(geometry))
            keys.append(key)
            
            
            #we need to initialize distance and time diff with 0 on the first position
            grp.loc[0,'dist']=0
            grp.loc[0,'time_difference']=0
            
            #identify decoupling location
            max_time = max(grp.time_difference)
            
            idx = list(grp.index[grp.time_difference==max_time])[0]
            
            #Sometimes the aircraft stays a loong time at the parking position, and that becomes the max. 
            #We need to check if the max is at index 1 and then check the max also without that position
            
            if idx ==1:
                max_time = max(grp.time_difference[2:])
                idx = list(grp.index[grp.time_difference==max_time])[0]
                
            decoupling_location = Point(grp.loc[idx,'latitude'],grp.loc[idx,'longitude'])    
            
            #flatten other info
            airline.append(grp.airline[0])
            day.append(grp.day[0])
            hour.append(grp.hour[0])
            month.append(grp.month[0])
            max_time_list.append(max_time)
            
            #green zone check
            green.append(green_zone_polygon.contains(decoupling_location))
            decoup_loc.append(decoupling_location)
            length.append(grp.iloc[0:idx]['dist'].sum())
            gdf = gpd.GeoDataFrame({'key': keys,
                                    'airline':airline,
                                    'day':day,
                                    'hour':hour,
                                    'month':month,
                                    'green':green,
                                    'decoup':decoup_loc,
                                    'max_time':max_time_list,
                                    'length':length,
                                    'geometry':geometries})
            gdf.crs= 4326
    
    gdf.reset_index(inplace=True,drop=True)

                                
    return gdf

In [25]:
#load the green_zone
green_gpd = gpd.read_file('greenzone.geojson')
green_gpd.crs = 4326
#extract it
green_zone = green_gpd['geometry'][0]

#rebuild (switch lat with lon)
green_zone_coords =  list(zip(green_zone.boundary.coords.xy[1],green_zone.boundary.coords.xy[0]))
green_zone_polygon = Polygon(green_zone_coords)

In [26]:
transformed_df= transform(data,green_zone)

In [27]:
transformed_df.head()

,key,airline,day,hour,month,green,decoup,max_time,length,geometry
0,789932628,TRA,5,4,2,True,POINT (52.30466 4.7648),226.0,0.016262,"LINESTRING (52.30541 4.76520, 52.30534 4.76499..."
1,789934547,TFL,5,4,2,True,POINT (52.31084 4.77007),188.0,0.265227,"LINESTRING (52.30981 4.76703, 52.30990 4.76730..."
2,789936916,TRA,5,4,2,True,POINT (52.31083 4.77081),257.0,0.073620,"LINESTRING (52.31003 4.77119, 52.31025 4.77121..."
3,789937787,TRA,5,5,2,False,POINT (52.30397 4.76779),178.0,0.099333,"LINESTRING (52.30465 4.76710, 52.30438 4.76730..."
4,789937866,SWR,5,5,2,True,POINT (52.30449 4.76422),280.0,0.053683,"LINESTRING (52.30512 4.76255, 52.30520 4.76278..."


In [28]:
#adding information about the gates, handlers and piers. 
#TODO this will be changed, now it is random
gates = random.choices(range(1, 60),k= len(transformed_df))
handlers = random.choices(['Dnata','Menzies','Swissport','WFSH','KLM Cargo','Aviapartner','VP Passenger'],k=len(transformed_df))
piers = random.choices(['A','B','C','D','E','F','G'],k=len(transformed_df))

transformed_df['pier'] = piers
transformed_df['gate'] = gates
transformed_df['handler'] = handlers

In [29]:
transformed_df.head()

,key,airline,day,hour,month,green,decoup,max_time,length,geometry,pier,gate,handler
0,789932628,TRA,5,4,2,True,POINT (52.30466 4.7648),226.0,0.016262,"LINESTRING (52.30541 4.76520, 52.30534 4.76499...",D,43,Dnata
1,789934547,TFL,5,4,2,True,POINT (52.31084 4.77007),188.0,0.265227,"LINESTRING (52.30981 4.76703, 52.30990 4.76730...",F,46,Menzies
2,789936916,TRA,5,4,2,True,POINT (52.31083 4.77081),257.0,0.073620,"LINESTRING (52.31003 4.77119, 52.31025 4.77121...",D,50,Aviapartner
3,789937787,TRA,5,5,2,False,POINT (52.30397 4.76779),178.0,0.099333,"LINESTRING (52.30465 4.76710, 52.30438 4.76730...",G,19,Swissport
4,789937866,SWR,5,5,2,True,POINT (52.30449 4.76422),280.0,0.053683,"LINESTRING (52.30512 4.76255, 52.30520 4.76278...",A,33,Swissport


In [30]:
transformed_df.to_csv('transformed_data.csv')